In [8]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                          11/2020     Adrien Poindron    #
###########################################################

# Imports

In [9]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [10]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path

In [11]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [12]:
from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from scipy.signal import savgol_filter

In [14]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/1.2/data_fct_Adrien_point_by_point

matplotlib.rcParams.update({'font.size': 25})

In [91]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.00e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichiers et chargement des données

In [44]:
def load_file_GUI(dir_string):

    root = tk.Tk()
    root.withdraw()

    file_path = filedialog.askopenfilename(initialdir = dir_string,
                                       multiple=True)
    return file_path

In [48]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint')
print(file_path)

('/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint/2.1/All_stat_DC01_RF08_50eV.npz', '/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint/2.1/All_stat_DC01_RF09_50eV.npz', '/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint/2.1/All_stat_DC01_RF10_50eV.npz', '/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint/2.1/All_stat_DC01_RF11_50eV.npz', '/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint/2.1/All_stat_DC03_RF08_50eV.npz', '/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint/2.1/All_stat_DC03_RF09_50eV.npz', '/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint/2.1/All_stat_DC03_RF10_50eV.npz', '/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint/2.1/All_stat_DC03_RF11_50eV.npz', '/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint/2.1/All_stat_DC05_RF08_50eV.npz', '/home/adrian/Docu

In [62]:
# load check
time         = []
Temperature  = []
Temp_var     = []
Fluorescence = []
Fluo_var     = []
SNR          = []
Ray          = []
Len          = []
DeltaEc      = []
condition    = []

for outfile in file_path:
    with load(outfile) as data:
        time.append(data['time'])
        Temperature.append(data['Temperature'])
        Temp_var.append(data['Temp_var'])
        Fluorescence.append(data['Fluo'])
        Fluo_var.append(data['Fluo_var'])
        SNR.append(data['SNR'])
        Ray.append(data['Ray'])
        Len.append(data['Len'])
        DeltaEc.append(data['DeltaEc'])
        condition.append(data['condition'])

# Temperature,Temp_var,
# Fluo, Fluo_var, SNR,
# Ray, Len,
# DeltaEc,
# condition

# Traitement des données

In [64]:
shape(time),shape(Temperature),shape(Fluorescence)

((28, 100, 20850), (28, 100, 20850, 3), (28, 100, 19860))

In [95]:
cond_num = len(condition)

plot_point = 10
plot_try   = 25

tt    = time[plot_point][plot_try]
T_all = Temperature[plot_point][plot_try]
PM    = Fluorescence[plot_point][plot_try]

print(shape(tt),shape(T_all),shape(PM))

n_dt = 100
dt  = 2*pi / (n_dt*Omega)
i_free__fly_Lan = 1000
i_laser_fly = 6000
i_laser_fly_quench = 8000
print(dt)
diff_time = len(tt)-len(PM)
print(diff_time)

(20850,) (20850, 3) (19860,)
5e-09
990


In [102]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Plot temperature Langevin + md laser  ALL SimuType  ALL TEMP MEAN
matplotlib.rcParams.update({'font.size': 25})

#fig_name = file_name[-9:]
figname = 'T_and_PM_plot_example'
fig1 = plt.figure(num=figname,clear='True')
fig1.set_size_inches(11.69,8.27)
# set height ratios for sublots
gs = gridspec.GridSpec(2, 1, height_ratios=[7, 3]) 
# the fisrt subplot
ax1 = plt.subplot(gs[0])

ax1.semilogy(tt*1e3,savgol_filter(mean(T_all,axis=1),25,1), label='Tx',color='red',lw=2.5)

time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-5,1e4,color='black')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-5,1e4,color='black',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='black',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
# ax1.vlines((ta[2])*1e3,1e-4,1e4,color='black')
# ax1.hlines(0.5,7.5,8.5, linestyle=':')
# ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
ylabel('T [K]')

ax1.annotate('1', xy=(0.22,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('2', xy=(2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('3', xy=((3.5+7.5)/2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
# ax1.annotate('4', xy=((7.5+7.57)/2,2.5e-4), xycoords='data',
# size=15, ha='center', va='top', color='xkcd:black',
# bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('5', xy=((7.5+10.5)/2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))

# annotate('Laser ON', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul')

# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

# ax2 = subplot(212,sharex=ax1)
ax2 = plt.subplot(gs[1], sharex = ax1)

# ax2.plot(tt_md*1e3,PM, label='Tx')

tt_1ms_step = tt[diff_time::2000]
PM_1ms = PM[::2000]
tt_1us_step = tt[diff_time::2]
PM_1us = PM[::2]

ax2.scatter(tt_1ms_step*1e3,PM_1ms, label='cts/ms',marker='P',s=200,lw=1.5,color='black',zorder=100)
ax2.plot(tt_1us_step[:-500]*1e3,PM_1us[:-500], label='cts/µs',color='blue')
ax2.legend()
time_switch = i_free__fly_Lan*dt
ax2.vlines(time_switch*1e3,1e-4,150,color='black')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
# ax2.vlines(time_switch*1e3,1e-4,150,color='black',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
# ax2.vlines(time_switch*1e3,1e-4,70,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
# ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,90,color='purple')
ax2.grid()
# ax2.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=20))
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time [ms]')
ylabel('PM signal')

left, bottom, width, height = [0.345, 0.565, 0.35, 0.38] # cadre zoom
axins = fig1.add_axes([left, bottom, width, height])

axins.semilogy(tt*1e3,mean(T_all,axis=1), label='Tx',color='red',lw=2.5,marker='x')
# axins.hlines(0.5,7.4,7.66, linestyle=':')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
axins.vlines(time_switch*1e3,1e-4,1e4,color='black',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly  +i_laser_fly_quench)*dt
axins.vlines(ta[2]*1e3,1e-4,1e4,color='black')

axins.add_patch(
     patches.Rectangle(
        (t_in*1000, 3e-4),
        (t_out - t_in)*1000,
        0.1,
        edgecolor ='k',
        facecolor =  cc('grey',0.5),
        fill=True
     ) )

x1, x2, y1, y2 = 7.44, 7.63, 0.00012, 50 # limites zoom
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

# axins.yaxis.tick_right()


locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
axins.yaxis.set_major_locator(locmaj)
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
axins.yaxis.set_minor_locator(locmin)
axins.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
plt.setp(axins.get_xticklabels(), backgroundcolor="w")
plt.setp(axins.get_yticklabels(), backgroundcolor="w")

axins.grid(b=True, which='major', linestyle='-')
axins.grid(b=True, which='minor', linestyle='--')

# axins.tick_params(axis="y",direction="in", pad=-53)
mark_inset(ax1, axins, loc1=1, loc2=3, fc="none", ec="0.25")

axins.annotate('3', xy=(7.48,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
axins.annotate('4', xy=((7.5+7.53)/2,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
axins.annotate('5', xy=(7.55,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))

ax1.set_xlim(-0.15, 10.75)
ax1.set_ylim(0.00005, 1000)
ax2.set_ylim(-0, 135)

plt.tight_layout()
subplots_adjust(hspace=0.0,bottom=0.11,left=0.12,right=0.98,top=0.97)
# savefig(fname = figname+'_'+re.sub('/','_',dirname_hc[31:-1])+'.png',dpi=300)
# savefig(figname+'_D'+str(Detuning)+'_50eV'+'.png',dpi=300)

NameError: name 'ta' is not defined